In [1]:
import pandas as pd
import numpy as np
from feature_aug import feature_aug


# 訓練集處理

In [3]:
training_df = pd.read_csv('/Users/wukeyang/ntu_course/2024-HTML/final_project/html-2024-fall-final-project-stage-1/train_data.csv')

In [4]:
# 從 home_team_season 提取賽季資訊
training_df['season'] = training_df['home_team_season'].str.extract('(\d{4})').astype(float)

# 對仍然缺失的值使用 away_team_season
mask = training_df['season'].isna()
training_df.loc[mask, 'season'] = training_df.loc[mask, 'away_team_season'].str.extract('(\d{4})').astype(float)

# 對剩餘的缺失值，使用 date 欄位的年份
mask = training_df['season'].isna()
training_df.loc[mask, 'season'] = pd.to_datetime(training_df.loc[mask, 'date']).dt.year

# 確認是否還有缺失值
print("剩餘缺失值數量：", training_df['season'].isna().sum())

剩餘缺失值數量： 0


In [5]:
# 計算每個欄位的缺失值數量及比例
missing_values = training_df.isna().sum()
missing_percentages = (training_df.isna().sum() / len(training_df)) * 100
missing_info = pd.DataFrame({
    '缺失值數量': missing_values,
    '缺失值比例(%)': missing_percentages
})

# 篩選出缺失值比例超過10%的欄位
high_missing = missing_info[missing_info['缺失值比例(%)'] > 10]

print("缺失值比例超過10%的欄位統計:")
print("-" * 50)
print(high_missing)
print("\n總資料筆數:", len(training_df))

缺失值比例超過10%的欄位統計:
--------------------------------------------------
                                      缺失值數量   缺失值比例(%)
home_pitcher_rest                      1534  13.861028
away_pitcher_rest                      1558  14.077889
home_pitcher_earned_run_avg_10RA       1274  11.511701
home_pitcher_SO_batters_faced_10RA     1271  11.484594
home_pitcher_H_batters_faced_10RA      1284  11.602060
home_pitcher_BB_batters_faced_10RA     1267  11.448450
away_pitcher_earned_run_avg_10RA       1293  11.683383
away_pitcher_SO_batters_faced_10RA     1277  11.538809
away_pitcher_H_batters_faced_10RA      1279  11.556881
away_pitcher_BB_batters_faced_10RA     1291  11.665311
home_pitcher_earned_run_avg_mean       1274  11.511701
home_pitcher_earned_run_avg_std        1998  18.053673
home_pitcher_earned_run_avg_skew       2692  24.324569
home_pitcher_SO_batters_faced_mean     1285  11.611096
home_pitcher_SO_batters_faced_std      1980  17.891027
home_pitcher_SO_batters_faced_skew     2672  24.1438

In [6]:
def fill_missing_values(df, col):
    if df[col].dtype not in ['int64', 'float64']:
        return df[col]
    
    df_copy = df.copy()
    
    # 按日期排序
    df_copy = df_copy.sort_values('date')
    
    #   使用比賽日期前的當季平均填補
    temp = df_copy.groupby('season').apply(
        lambda x: x[col].fillna(
            x[col].expanding().mean()
        )
    )
    df_copy[col] = temp.reset_index(level=0, drop=True)
    
    #   若還有缺失值,使用前一季的整季平均值
    if df_copy[col].isna().any():
        # 計算每個賽季的平均值
        season_means = df_copy.groupby('season')[col].mean()
        
        for season in df_copy['season'].unique():
            mask = (df_copy['season'] == season) & df_copy[col].isna()
            if mask.any() and season > df_copy['season'].min():
                prev_season_mean = season_means.get(season - 1)
                if pd.notna(prev_season_mean):
                    df_copy.loc[mask, col] = prev_season_mean
    
    #   若仍有缺失值(例如第一季),則填補0.5或0
    if df_copy[col].isna().any():
        if 'rate' in col.lower() or 'avg' in col.lower() or 'percentage' in col.lower():
            df_copy[col] = df_copy[col].fillna(0.5)
        else:
            df_copy[col] = df_copy[col].fillna(0)
            
    return df_copy[col]

#   先找出需要填補的數值型欄位
numeric_cols = training_df.select_dtypes(include=['int64', 'float64']).columns

#   對每個欄位進行填補
for col in numeric_cols:
    try:
        training_df[col] = fill_missing_values(training_df, col)
    except Exception as e:
        print(f"處理欄位 {col} 時發生錯誤: {str(e)}")


In [7]:
missing_values = training_df.isna().sum()
print("以下欄位還有缺失值:")
print(missing_values[missing_values > 0])

以下欄位還有缺失值:
is_night_game       553
home_pitcher        553
away_pitcher        553
home_team_season    553
away_team_season    553
dtype: int64


In [8]:
#   先處理 is_night_game
probabilities = training_df['is_night_game'].value_counts(normalize=True)

# 使用 numpy.random.choice 根據機率分布隨機填充
training_df['is_night_game'] = training_df['is_night_game'].fillna(
    pd.Series(
        np.random.choice(
            probabilities.index,  # 可能的值 [True, False]
            size=training_df['is_night_game'].isna().sum(),  # 需要填充的數量
            p=probabilities.values  # 每個值的機率
        ),
        index=training_df[training_df['is_night_game'].isna()].index  # 確保填充到正確的位置
    )
)

#   刪除 pitcher 相關缺失值的行
training_df.dropna(subset=['home_pitcher', 'away_pitcher'], inplace=True)
training_df.drop(["home_team_season", "away_team_season","id"], axis=1, inplace=True)

In [9]:
training_df['date'] = pd.to_datetime(training_df['date']) 
training_df = feature_aug(training_df)

training_df['month'] = training_df['date'].dt.month
training_df['day'] = training_df['date'].dt.day


# 測試集處理

In [9]:
test_df = pd.read_csv("data/same_season_test_data.csv")

In [10]:
test_df["home_team_season"]

0       HOU_2017
1       LAA_2023
2       STL_2016
3       DET_2019
4       ATL_2016
          ...   
6180    CHW_2022
6181    BOS_2018
6182    LAA_2017
6183    TBR_2018
6184    NYY_2022
Name: home_team_season, Length: 6185, dtype: object

In [11]:
# 計算每個欄位的缺失值數量及比例
missing_values = test_df.isna().sum()
missing_percentages = (test_df.isna().sum() / len(test_df)) * 100
missing_info = pd.DataFrame({
    '缺失值數量': missing_values,
    '缺失值比例(%)': missing_percentages
})

# 篩選出缺失值比例超過10%的欄位
high_missing = missing_info[missing_info['缺失值比例(%)'] > 10]

print("缺失值比例超過10%的欄位統計:")
print("-" * 50)
print(high_missing)
print("\n總資料筆數:", len(test_df))

缺失值比例超過10%的欄位統計:
--------------------------------------------------
Empty DataFrame
Columns: [缺失值數量, 缺失值比例(%)]
Index: []

總資料筆數: 6185


In [12]:
# 從 home_team_season 提取賽季資訊
test_df['season'] = test_df['home_team_season'].str.extract('(\d{4})').astype(float)

# 對仍然缺失的值使用 away_team_season
mask = test_df['season'].isna()
test_df.loc[mask, 'season'] = test_df.loc[mask, 'away_team_season'].str.extract('(\d{4})').astype(float)

# 如果還有缺失值，才使用隨機填充
missing_mask = test_df['season'].isna()
if missing_mask.any():
    test_df.loc[missing_mask, 'season'] = np.random.choice(
        range(2016, 2024), 
        size=missing_mask.sum()
    )

# 將 season 轉換為日期
test_df['date'] = pd.to_datetime(test_df['season'].astype(int).apply(lambda x: f"{x}-08-01"))

In [13]:
#   先找出需要填補的數值型欄位
numeric_cols = test_df.select_dtypes(include=['int64', 'float64']).columns

#   對每個欄位進行填補
for col in numeric_cols:
    try:
        test_df[col] = fill_missing_values(test_df, col)
    except Exception as e:
        print(f"處理欄位 {col} 時發生錯誤: {str(e)}")

missing_values = test_df.isna().sum()
print("以下欄位還有缺失值:")
print(missing_values[missing_values > 0])

以下欄位還有缺失值:
is_night_game       309
home_pitcher        309
away_pitcher        309
home_team_season    309
away_team_season    309
dtype: int64


In [14]:
#   先處理 is_night_game
probabilities = test_df['is_night_game'].value_counts(normalize=True)

# 使用 numpy.random.choice 根據機率分布隨機填充
test_df['is_night_game'] = test_df['is_night_game'].fillna(
    pd.Series(
        np.random.choice(
            probabilities.index,  # 可能的值 [True, False]
            size=test_df['is_night_game'].isna().sum(),  # 需要填充的數量
            p=probabilities.values  # 每個值的機率
        ),
        index=test_df[test_df['is_night_game'].isna()].index  # 確保填充到正確的位置
    )
)

#   刪除 pitcher 相關缺失值的行
test_df.dropna(subset=['home_pitcher', 'away_pitcher'], inplace=True)
test_df.drop(["home_team_season", "away_team_season"], axis=1, inplace=True)

In [15]:
# 確保日期欄位是datetime格式
test_df['date'] = pd.to_datetime(test_df['date']) 

test_df = feature_aug(test_df, training_data=training_df)

training_df['month'] = training_df['date'].dt.month
training_df['day'] = training_df['date'].dt.day